In [2]:
from requests import get
from pandas import DataFrame, concat, to_datetime, Timedelta, json_normalize
from time import time
#import numpy as np
#from seaborn import set, distplot
#import matplotlib.pyplot as plt
#from math import ceil
from plotly.graph_objects import Scatter
import plotly.figure_factory as ff
from plotly.express import colors
from dotenv import load_dotenv
from datetime import datetime
from time import time
import json, os

In [3]:
fetchCupData = True

lastUpdatedInDays = 1

In [4]:
# current time
currentTime = time()
timeToCheckAgainst = datetime.fromtimestamp(currentTime - lastUpdatedInDays*24*60*60)

if not os.path.exists('../data files'):
    os.makedirs('../data files')

print('Checking for cup data...')
if os.path.exists("../data files/rawCupData.json"):
    cupDataTime = datetime.fromtimestamp(os.path.getmtime('../data files/rawCupData.json'))
    print('Found cup data from:', cupDataTime.strftime('%c'))
    
    # if older than 'lastUpdatedInDays' days, fetch again
    if cupDataTime <= timeToCheckAgainst:
        print('Locally saved data is older than {} hours!'.format(lastUpdatedInDays*24))
        print('Have to fetch again...')
        fetchCupData = True
    else:
        print('Will be using this locally saved data...')
        fetchCupData = False
else:
    print('No local data found!')
    print('Have to fetch again...')
    fetchCupData = True

Checking for cup data...
Found cup data from: Sat Sep 26 17:17:19 2020
Locally saved data is older than 24 hours!
Have to fetch again...


In [5]:
load_dotenv()

domain = os.getenv("URL")

if fetchCupData:
    print('Getting Cup data... Please wait!')
    startTime = time()

    # get cup data
    response = get('https://' + domain + 'cup')
    cupData = response.json()

    print('Took', round(time() - startTime, 2), 'seconds to get Cup data.')
    
    # save locally for later reuse
    with open('../data files/rawCupData.json', 'w') as fp:
        json.dump(cupData, fp, indent=4)
else:
    # use locally saved data
    with open('../data files/rawCupData.json', 'r') as fp:
        cupData = json.load(fp)

Getting Cup data... Please wait!
Took 15.31 seconds to get Cup data.


In [12]:
cupDF = DataFrame.from_dict(cupData, orient='columns')
cupDF = cupDF.drop(['size', 'updated_at', 'created_at_melbourne_date_time', 'updated_at_melbourne_date_time'], axis = 1)
df = cupDF.groupby('status').size().reset_index(name='count')
df

,status,count
0,B,828
1,R,4985


In [14]:
df

,status,count
0,B,828
1,R,4985


In [16]:
df.to_csv('../data files/borrowReturnCounts.csv', index = False)